In [1]:
# Remember to adjust your student ID in meta.xml
import numpy as np
import pickle
import random
import gym
from simple_custom_taxi_env import SimpleTaxiEnv
from tqdm import tqdm

c:\Users\qwer9\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\envs\registration.py:307: DeprecationWarning: The package name gym_minigrid has been deprecated in favor of minigrid. Please uninstall gym_minigrid and install minigrid with `pip install minigrid`. Future releases will be maintained under the new package name minigrid.
  fn()


In [ ]:
# def table_q_learning(episode = 5000, alpha = 0.1, gamma = 0.99, epsilon_start = 1.0, epsilon_end = 0.1, epsilon_decay = 0.999):
#     q_table = {}
#     total_rewards = []

#     for i in tqdm(range(episode)):
#         env = SimpleTaxiEnv(fuel_limit=5000)
#         raw_state, _ = env.reset()
#         total_reward = 0
#         done = False
#         step_count = 0
#         epsilon = epsilon_start
#         taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = raw_state
#         taxi_pos = (taxi_row, taxi_col)
#         state = ( obstacle_south, obstacle_north, obstacle_east, obstacle_west)

#         saved = False
#         # if (i + 1) % 10000 == 0:
#         #     saved = True
#         #     print(f"=========================={i+1}th episodes Start!==========================")
        
#         while not done:
#             if state not in q_table:
#                 q_table[state] = np.zeros(6)
#             if random.uniform(0, 1) < epsilon:
#                 action = random.choice([0, 1, 2, 3, 4, 5])
#             else:
#                 action = np.argmax(q_table[state])
#             next_raw_state, reward, done, _ = env.step(action)

#             if saved:
#                 env.render_env(taxi_pos, action = action, step = step_count)

#             taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = next_raw_state
#             taxi_pos = (taxi_row, taxi_col)
#             next_state = ( obstacle_south, obstacle_north, obstacle_east, obstacle_west)
#             if next_state not in q_table:
#                 q_table[next_state] = np.zeros(6)
#             q_table[state][action] = q_table[state][action] + alpha*(reward + gamma*np.max(q_table[next_state]) - q_table[state][action])
#             state = next_state
#             total_reward += reward
#             step_count += 1
#             if epsilon > epsilon_end:
#                 epsilon *= epsilon_decay
#             # if step_count > 500:
#             #     break

#         total_rewards.append(total_reward)
#         if saved:
#             print(f"Episode: {i + 1}/{episode}, Total Reward: {total_reward}, Steps: {step_count}, Epsilon: {epsilon}")

#     return q_table, total_rewards

# def save_q_table(q_table):
#     with open('q_table.pkl', 'wb') as f:
#         pickle.dump(q_table, f)


In [ ]:
def table_q_learning(episode = 5000, alpha = 0.1, gamma = 0.99, epsilon_start = 1.0, epsilon_end = 0.1, epsilon_decay = 0.999):
    q_table = {}
    total_rewards = []

    for i in tqdm(range(episode)):
        env = SimpleTaxiEnv(fuel_limit=5000)
        raw_state, _ = env.reset()
        total_reward = 0
        done = False
        step_count = 0
        epsilon = epsilon_start
        taxi_row, taxi_col, R_x, R_y, G_x, G_y, Y_x, Y_y, B_x, B_y, obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = raw_state
        taxi_pos = (taxi_row, taxi_col)

        station = []
        station.append((R_x, R_y))
        station.append((G_x, G_y))
        station.append((Y_x, Y_y))
        station.append((B_x, B_y))

        progress = 0
        goal = station[progress]
        relative_goal_pos = (goal[0] - taxi_row, goal[1] - taxi_col)

        state = ( obstacle_south, obstacle_north, obstacle_east, obstacle_west, relative_goal_pos[0], relative_goal_pos[1])

        saved = False
        # if (i + 1) % 10000 == 0:
        #     saved = True
        #     print(f"=========================={i+1}th episodes Start!==========================")
        
        while not done:
            if state not in q_table:
                q_table[state] = np.zeros(6)
            if random.uniform(0, 1) < epsilon:
                action = random.choice([0, 1, 2, 3, 4, 5])
            else:
                action = np.argmax(q_table[state])
            next_raw_state, reward, done, _ = env.step(action)

            if saved:
                env.render_env(taxi_pos, action = action, step = step_count)

            taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = next_raw_state

            if (taxi_row, taxi_col) == goal:
                progress = (progress + 1) % 4
                goal = station[progress]
                reward += 10
            else:
                reward -= 0.5 * (abs(taxi_row - goal[0]) + abs(taxi_col - goal[1]))

            relative_goal_pos = (goal[0] - taxi_row, goal[1] - taxi_col)
            next_state = ( obstacle_south, obstacle_north, obstacle_east, obstacle_west, relative_goal_pos[0], relative_goal_pos[1])

            if next_state not in q_table:
                q_table[next_state] = np.zeros(6)
            q_table[state][action] = q_table[state][action] + alpha*(reward + gamma*np.max(q_table[next_state]) - q_table[state][action])
            state = next_state
            total_reward += reward
            step_count += 1
            if epsilon > epsilon_end:
                epsilon *= epsilon_decay
            # if step_count > 500:
            #     break

        total_rewards.append(total_reward)
        if saved:
            print(f"Episode: {i + 1}/{episode}, Total Reward: {total_reward}, Steps: {step_count}, Epsilon: {epsilon}")

    return q_table, total_rewards

def save_q_table(q_table):
    with open('q_table.pkl', 'wb') as f:
        pickle.dump(q_table, f)


In [21]:
q_table, total_rewards = table_q_learning(episode = 5000, alpha = 0.1, gamma = 0.99, epsilon_start = 1.0, epsilon_end = 0.1, epsilon_decay = 0.999)
save_q_table(q_table)

100%|██████████| 5000/5000 [04:36<00:00, 18.11it/s]
